<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Trial-Requests" data-toc-modified-id="Trial-Requests-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Trial Requests</a></span><ul class="toc-item"><li><span><a href="#General-Request" data-toc-modified-id="General-Request-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>General Request</a></span></li><li><span><a href="#Getting-Structure" data-toc-modified-id="Getting-Structure-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Getting Structure</a></span><ul class="toc-item"><li><span><a href="#Request-Data" data-toc-modified-id="Request-Data-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Request Data</a></span></li><li><span><a href="#Extract-Structural-Info-and-Build-ASE-Atoms-Object" data-toc-modified-id="Extract-Structural-Info-and-Build-ASE-Atoms-Object-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Extract Structural Info and Build ASE Atoms Object</a></span></li><li><span><a href="#Add-Atoms" data-toc-modified-id="Add-Atoms-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Add Atoms</a></span></li><li><span><a href="#Get-POTCAR-Parameters" data-toc-modified-id="Get-POTCAR-Parameters-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Get POTCAR Parameters</a></span></li></ul></li></ul></li><li><span><a href="#Functions" data-toc-modified-id="Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Collect-Bench-Set" data-toc-modified-id="Collect-Bench-Set-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Collect Bench Set</a></span></li></ul></div>

# Imports

In [9]:
import requests, ase
import numpy as np
import pickle as pkl

# Trial Requests

## General Request

In [10]:
url = (
'https://www.materialsproject.org/rest/v2/materials/mp-1234/vasp?API_KEY=rNLdA4JmwdI5kSuHHr1r'
)
response = requests.get(url)
data = response.json()['response'][0] # Unpack the data directly

In [11]:
energy = data['energy']
bandgap = data['band_gap']

## Getting Structure

### Request Data

In [12]:
url = ( 
'https://www.materialsproject.org/rest/v2/materials/mp-804/vasp/final_structure?API_KEY=rNLdA4JmwdI5kSuHHr1r'
)
response = requests.get(url)
data = response.json()['response'][0]


### Extract Structural Info and Build ASE Atoms Object

In [13]:
structure = data['final_structure']
cell = structure['lattice']
cell
a = cell['a']
b = cell['b']
c = cell['c']
alpha = cell['alpha']
beta = cell['beta']
gamma = cell['gamma']
volume = cell['volume']

system = ase.Atoms()
system.set_cell([a,b,c,alpha,beta,gamma])

### Add Atoms

In [14]:
sites = structure['sites']

for site in sites:
    species = site['label']
    pos = site['xyz']
    magmom = site['properties']['magmom']
    system.append(ase.Atom(species,pos,magmom=magmom))


### Get POTCAR Parameters

In [15]:
url = ( 
'https://www.materialsproject.org/rest/v2/tasks/mp-804/potcar?API_KEY=rNLdA4JmwdI5kSuHHr1r'
)
response = requests.get(url)
data = response.json()['response'][0]

potcar = data['potcar']

In [16]:
setups = {}
for setup in [i.split("_") for i in potcar]:
    if len(setup) > 1:
        setups[setup[0]] = '_'+setup[1]
    else:
        setups[setup[0]] = ''
        
print(setups)

{'N': '', 'Ga': '_d'}


# Functions

In [17]:
def build_cell(structure):
    cell = structure['lattice']
    a = cell['a']
    b = cell['b']
    c = cell['c']
    alpha = cell['alpha']
    beta = cell['beta']
    gamma = cell['gamma']
    volume = cell['volume']

    system = ase.Atoms()
    system.set_cell([a,b,c,alpha,beta,gamma])
    system.set_pbc(True)

    # Add atoms
    sites = structure['sites']

    for site in sites:
        species = site['label']
        pos = site['xyz']
        try:
            magmom = site['properties']['magmom']
        except KeyError:
            magmom = 0.
        system.append(ase.Atom(species,pos,magmom=magmom))
        
    return system

def do_request(_id):
    # Setup query urls
    url_head = 'https://www.materialsproject.org/rest/v2/'
    url_tail = '?API_KEY=rNLdA4JmwdI5kSuHHr1r'
    url_structure_initial = 'materials/mp-{}/vasp/initial_structure'.format(_id)
    url_structure_final = 'materials/mp-{}/vasp/final_structure'.format(_id)
    url_bandgap = 'materials/mp-{}/vasp/'.format(_id)
    url_potcar = 'tasks/mp-{}/potcar'.format(_id)

    # Initial Structure Query and Extraction
    url = url_head+url_structure_initial+url_tail
    response = requests.get(url)
    try:
        data = response.json()['response'][0]
        
        # Build cell
        structure = data['initial_structure']
        system_initial = build_cell(structure)
            
    except IndexError:
        #print('ID {} not valid.'.format(id))
        return None# Don't do Potcar query, bc id not valid
    
    # Final Structure Query and Extraction
    url = url_head+url_structure_final+url_tail
    response = requests.get(url)
    try:
        data = response.json()['response'][0]
        
        # Build cell
        structure = data['final_structure']
        system_final = build_cell(structure)
            
    except IndexError:
        #print('ID {} not valid.'.format(id))
        return None# Don't do Potcar query, bc id not valid

    # Potcar Query and Extraction
    url = url_head+url_potcar+url_tail
    response = requests.get(url)
    try:
        data = response.json()['response'][0]
        potcar = data['potcar']
        setups = {}
        for setup in [i.split("_") for i in potcar]:
            if len(setup) > 1:
                setups[setup[0]] = '_'+setup[1]
            else:
                setups[setup[0]] = ''
    except IndexError:
        print('Unable to find POTCAR information for with ID {}.'.format(id))
        return None
    
    # Get Band Gap 
    url = url_head+url_bandgap+url_tail
    response = requests.get(url)
    data = response.json()['response'][0]
    bandgap = data['band_gap']
        
    return system_initial, system_final, setups, bandgap

def collect_bench_set(_id):
    key = 'mp-{}'.format(_id)
    try:
        system_initial, system_final, setups, bandgap = do_request(_id)
        entry = {'structure_initial':system_initial,'structure_final':system_final,'potcar':setups,'bandgap':bandgap}
    except TypeError:
        entry = {'error': 'Bad ID'}
    
    return key,entry

# Collect Bench Set

In [ ]:
bench_set = {}
ids = np.arange(900)
errors = []

for _id in ids:
    if _id%25 == 0:
        print('{} of {}'.format(_id,len(ids)))
    try:
        key,entry = collect_bench_set(_id)
        bench_set[key] = entry
    except:
        errors.append(_id)


In [ ]:
with open('matproj_bench_set_0_to_900.pkl','wb') as fout:
    pkl.dump(bench_set,fout)

In [5]:
bad_ids = [key for key in bench_set.keys() if bench_set[key] == {'error': 'Bad ID'}]
len(bad_ids)

NameError: name 'bench_set' is not defined

In [3]:
errors = []
ids = np.arange(900,1400)
bench_set2 = {}

for _id in ids:
    if _id%25 == 0:
        print('{} of {}'.format(_id,900+len(ids)))
    try:
        key,entry = collect_bench_set(_id)
        bench_set2[key] = entry
    except:
        errors.append(_id)
        


900 of 1400
925 of 1400
950 of 1400
975 of 1400
1000 of 1400
1025 of 1400
1050 of 1400
1075 of 1400
1100 of 1400
1125 of 1400
1150 of 1400
1175 of 1400
1200 of 1400
1225 of 1400
1250 of 1400
1275 of 1400
1300 of 1400
1325 of 1400
1350 of 1400
1375 of 1400


errors

In [4]:
bad_ids2 = [key for key in bench_set2.keys() if bench_set2[key] == {'error': 'Bad ID'}]
len(bad_ids2)

128

In [7]:
with open('matproj_bench_set_900_to_1400.pkl','wb') as fout:
    pkl.dump(bench_set2,fout)

In [8]:
ls *pkl

matproj_bench_set_0_to_900.pkl     test.pkl
matproj_bench_set_900_to_1400.pkl
